In [3]:
!pip install pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 893.7/893.7 kB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 853.6/853.6 kB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 5.9 MB/s eta 0:00:00
   ━━

In [8]:
!apt-get update && apt-get install -y libsndfile1 ffmpeg
!pip install Cython packaging
!pip install 'git+https://github.com/NVIDIA/NeMo.git@main#egg=nemo_toolkit[asr]'


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,378 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntu

In [7]:


import os
import csv
import time
import torch
import logging
from pyannote.core import Annotation, Segment
from pyannote.metrics.diarization import DiarizationErrorRate

# --- 1. IMPORTATION PROPRE DE NEMO ---
logging.getLogger("nemo_logger").setLevel(logging.ERROR)
from nemo.collections.asr.models import SortformerEncLabelModel

# --- 2. CONFIGURATION ---
AUDIO_FOLDER = "/content/drive/MyDrive/GroundTruth_clean_no_overlap/audioMono"
GT_RTTM_FOLDER = "/content/drive/MyDrive/GroundTruth_clean_no_overlap/rttm"

OUTPUT_DIR = "results_nemo_sortformer"
CSV_OUTPUT_PATH = os.path.join(OUTPUT_DIR, "resultats_nemo_sortformer.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

COLLAR = 0.25
SKIP_OVERLAP = False

# --- 3. CHARGEMENT DU MODÈLE ---
print("⏳ Chargement du modèle NVIDIA Sortformer...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Appareil de calcul détecté : {device}")

try:
    model = SortformerEncLabelModel.from_pretrained(model_name="nvidia/diar_streaming_sortformer_4spk-v2")
    model.to(device)
    model.eval()
    print("✅ Modèle chargé avec succès.\n")
except Exception as e:
    print(f"❌ Erreur lors du chargement : {e}")
    exit()

# --- 4. FONCTIONS UTILITAIRES ---
def load_rttm_file(file_path):
    """Charge un fichier RTTM et le convertit en objet Pyannote."""
    annotation = Annotation()
    if not os.path.exists(file_path):
        return annotation
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 8 and parts[0] == 'SPEAKER':
                start = float(parts[3])
                duration = float(parts[4])
                label = parts[7]
                annotation[Segment(start, start + duration)] = label
    return annotation

def parse_sortformer_output(nemo_output):
    """Convertit la sortie texte (RTTM) de Sortformer v2 en objet Pyannote."""
    annotation = Annotation()

    # NeMo renvoie souvent un tuple ou une liste pour le batch. On extrait le texte.
    if isinstance(nemo_output, tuple):
        nemo_output = nemo_output[0]
    if isinstance(nemo_output, list) and len(nemo_output) > 0:
        nemo_output = nemo_output[0]

    # Si la sortie est bien le texte brut format RTTM
    if isinstance(nemo_output, str):
        lines = nemo_output.strip().split('\n')
        for line in lines:
            if not line.strip():
                continue
            parts = line.strip().split()
            # On vérifie qu'il s'agit bien d'une ligne RTTM valide
            if len(parts) >= 8 and parts[0] == 'SPEAKER':
                start = float(parts[3])
                duration = float(parts[4])
                speaker = parts[7]
                annotation[Segment(start, start + duration)] = speaker

    return annotation

# --- 5. INFÉRENCE ET ÉVALUATION ---
audio_files = sorted([f for f in os.listdir(AUDIO_FOLDER) if f.endswith('.wav')])

if not audio_files:
    print("❌ Aucun fichier audio (.wav) trouvé !")
    exit()

print(f"🚀 Lancement de la diarisation sur {len(audio_files)} fichiers...")
print("-" * 90)
print(f"{'FICHIER':<30} | {'DER (%)':<10} | {'VRAI':<5} | {'PRED':<5} | {'DIFF':<5} | {'TEMPS (s)'}")
print("-" * 90)

der_metric = DiarizationErrorRate(collar=COLLAR, skip_overlap=SKIP_OVERLAP)
results_data = []

for filename in audio_files:
    audio_path = os.path.join(AUDIO_FOLDER, filename)
    file_id = os.path.splitext(filename)[0]
    gt_path = os.path.join(GT_RTTM_FOLDER, file_id + ".rttm")

    if not os.path.exists(gt_path):
        continue

    # 1. Charger la vérité terrain
    reference = load_rttm_file(gt_path)

    # 2. Inférence Sortformer (Avec mesure du temps)
    start_time = time.time()
    try:
        raw_prediction = model.diarize(audio=[audio_path], batch_size=1)

        # --- BLOC DE DÉBOGAGE (À SUPPRIMER PLUS TARD) ---
        if filename == audio_files[0]: # On ne l'affiche que pour le premier fichier pour ne pas polluer
            print("\n" + "="*50)
            print(f"🕵️ DEBUG TYPE : {type(raw_prediction)}")
            print(f"🕵️ DEBUG CONTENU : {raw_prediction}")
            print("="*50 + "\n")
        # ------------------------------------------------

        hypothesis = parse_sortformer_output(raw_prediction)
    except Exception as e:
        print(f"❌ Erreur inférence sur {filename}: {e}")
        continue

    # 3. Métriques
    ref_end = reference.get_timeline().extent().end if reference else 0
    hyp_end = hypothesis.get_timeline().extent().end if hypothesis else 0
    uem_end = max(ref_end, hyp_end)
    uem_end = uem_end if uem_end > 0 else 1.0

    try:
        der_val = der_metric(reference, hypothesis, uem=Segment(0, uem_end)) * 100
    except:
        der_val = 100.0

    true_s = len(reference.labels())
    pred_s = len(hypothesis.labels())
    diff = pred_s - true_s

    results_data.append({
        'Filename': filename,
        'DER': der_val,
        'True_Speakers': true_s,
        'Pred_Speakers': pred_s,
        'Diff': diff,
        'Time_sec': infer_time
    })

    print(f"{filename:<30} | {der_val:>7.2f} % | {true_s:<5} | {pred_s:<5} | {diff:>4d} | {infer_time:>6.2f}s")

# --- 6. EXPORTATION DES RÉSULTATS ---
if results_data:
    print("\n" + "="*50)
    global_der = abs(der_metric) * 100
    print(f"🏆 DER MOYEN GLOBAL : {global_der:.2f} %")

    with open(CSV_OUTPUT_PATH, mode='w', newline='') as csv_file:
        fieldnames = ['Filename', 'DER', 'True_Speakers', 'Pred_Speakers', 'Diff', 'Time_sec']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        for data in results_data:
            writer.writerow({
                'Filename': data['Filename'],
                'DER': f"{data['DER']:.4f}",
                'True_Speakers': data['True_Speakers'],
                'Pred_Speakers': data['Pred_Speakers'],
                'Diff': data['Diff'],
                'Time_sec': f"{data['Time_sec']:.2f}"
            })
    print(f"📁 Fichier sauvegardé : {CSV_OUTPUT_PATH}")

⏳ Chargement du modèle NVIDIA Sortformer...
🖥️ Appareil de calcul détecté : cuda


[NeMo W 2026-02-24 13:21:00 dataloader:826] The following configuration keys are ignored by Lhotse dataloader: soft_label_thres,num_spks,session_len_sec


✅ Modèle chargé avec succès.

🚀 Lancement de la diarisation sur 60 fichiers...
------------------------------------------------------------------------------------------
FICHIER                        | DER (%)    | VRAI  | PRED  | DIFF  | TEMPS (s)
------------------------------------------------------------------------------------------


Diarizing: 0it [00:00, ?it/s]
Streaming Steps: 100%|██████████| 2/2 [00:00<00:00, 17.20it/s]
Diarizing: 1it [00:00,  4.57it/s]
[NeMo W 2026-02-24 13:21:00 dataloader:826] The following configuration keys are ignored by Lhotse dataloader: soft_label_thres,num_spks,session_len_sec



🕵️ DEBUG TYPE : <class 'list'>
🕵️ DEBUG CONTENU : [['0.320 3.600 speaker_0', '9.600 12.080 speaker_0', '15.040 17.360 speaker_0', '18.000 18.640 speaker_0', '21.360 24.160 speaker_0', '24.800 25.030 speaker_0', '6.080 8.400 speaker_1', '12.720 15.040 speaker_1', '18.880 21.520 speaker_1']]

clean_audio_000.wav            |  100.00 % | 2     | 0     |   -2 |   0.21s


...

gold_stereo_027.wav            |  100.00 % | 2     | 0     |   -2 |   0.21s


Diarizing: 0it [00:00, ?it/s]
Streaming Steps: 100%|██████████| 2/2 [00:00<00:00, 22.39it/s]
Diarizing: 1it [00:00,  6.07it/s]


gold_stereo_028.wav            |  100.00 % | 2     | 0     |   -2 |   0.21s


[NeMo W 2026-02-24 13:21:13 dataloader:826] The following configuration keys are ignored by Lhotse dataloader: soft_label_thres,num_spks,session_len_sec
Diarizing: 0it [00:00, ?it/s]
Streaming Steps: 100%|██████████| 2/2 [00:00<00:00, 22.51it/s]
Diarizing: 1it [00:00,  6.09it/s]


gold_stereo_029.wav            |  100.00 % | 2     | 0     |   -2 |   0.21s

🏆 DER MOYEN GLOBAL : 100.00 %
📁 Fichier sauvegardé : results_nemo_sortformer/resultats_nemo_sortformer.csv


In [5]:
from huggingface_hub import notebook_login
notebook_login()



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
from nemo.collections.asr.models import SortformerEncLabelModel

# load model from Hugging Face model card directly (You need a Hugging Face token)
diar_model = SortformerEncLabelModel.from_pretrained("nvidia/diar_streaming_sortformer_4spk-v2")


[NeMo W 2026-02-24 13:04:04 nemo_logging:364] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
      m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
    
[NeMo W 2026-02-24 13:04:04 nemo_logging:364] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
      m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
    
[NeMo W 2026-02-24 13:04:04 nemo_logging:364] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(flt)p?( \(default\))?$', token):
    
[NeMo W 2026-02-24 13:04:04 nemo_logging:364] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
      elif re.match('(dbl)p?( \(default\))?$', token):
    


diar_streaming_sortformer_4spk-v2.nemo:   0%|          | 0.00/471M [00:00<?, ?B/s]

[NeMo W 2026-02-24 13:04:14 modelPT:176] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    num_spks: 4
    session_len_sec: 90
    soft_label_thres: 0.5
    soft_targets: false
    labels: null
    batch_size: 4
    shuffle: true
    num_workers: 18
    validation_mode: false
    use_lhotse: false
    use_bucketing: false
    pin_memory: true
    window_stride: 0.01
    subsampling_factor: 8
    
[NeMo W 2026-02-24 13:04:14 modelPT:183] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: null
    is_tarred: false
    tarred_audio_filepaths: null
    sample_rate: 16000
    num_spks: 4
    sess

[NeMo I 2026-02-24 13:04:17 save_restore_connector:286] Model SortformerEncLabelModel was successfully restored from /root/.cache/huggingface/hub/models--nvidia--diar_streaming_sortformer_4spk-v2/snapshots/6dbf0d69730bfee097056692b86525a0a23b32f9/diar_streaming_sortformer_4spk-v2.nemo.
